In [55]:
from Bio import SeqIO, AlignIO
from Bio.Align import MultipleSeqAlignment
from collections import Counter
from sklearn.externals import joblib

import custom_funcs as cf
import pandas as pd
import numpy as np

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
pol = [s for s in SeqIO.parse('HIV-1_pol.fasta', 'fasta')]
len(pol)

159134

In [18]:
lengths = Counter([len(s.seq) for s in pol])
mc_len = lengths.most_common(1)[0][0]

In [19]:
# Filter to only the "most common" lengths.
pol_filtered = [s for s in pol if len(s.seq) == mc_len]
len(pol_filtered)

23163

In [71]:
str(pol_filtered[0].seq)

'PQITLWQRPLVAIKIGGQIKEALLDTGADDTVLEEMNLPGRWKPKMIGGIGGFIKVRQYDQIPIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF'

In [72]:
# Load binarizer
alignment = MultipleSeqAlignment(pol_filtered)
alignment = pd.DataFrame(np.array([list(s) for s in alignment]))
nonconserved_cols = cf.identify_nonconserved_cols(alignment, [])
alignment = cf.drop_conserved_cols(alignment, [], nonconserved_cols)
alignment = cf.drop_ambiguous_sequences(alignment, nonconserved_cols)
alignment = alignment.replace({'X':np.nan})
alignment.dropna(inplace=True)
alignment = alignment.ix[:, 2:]
alignment

,2,3,4,5,6,7,8,9,10,11,...,89,90,91,92,93,94,95,96,97,98
0,I,T,L,W,Q,R,P,L,V,A,...,L,T,Q,I,G,C,T,L,N,F
1,I,T,L,W,Q,R,P,L,V,A,...,L,T,Q,I,G,C,T,L,N,F
2,I,T,L,W,Q,R,P,L,V,T,...,L,T,Q,I,G,C,T,L,N,F
3,I,T,L,W,Q,R,P,L,V,T,...,L,T,Q,I,G,C,T,L,N,F
4,I,T,L,W,Q,R,P,R,V,T,...,L,T,Q,L,G,C,T,L,N,F
5,I,T,L,W,Q,R,P,I,V,T,...,L,T,Q,I,G,C,T,L,N,F
6,I,T,L,W,Q,R,P,I,V,T,...,M,T,Q,I,G,C,T,L,N,F
7,I,T,L,W,Q,R,P,I,V,T,...,M,T,Q,L,G,C,T,L,N,F
8,I,T,L,W,Q,R,P,L,V,T,...,L,T,Q,I,G,C,T,L,N,F
9,I,T,L,W,Q,R,P,L,V,T,...,M,T,Q,I,G,C,T,L,N,F


In [67]:
Counter(alignment[2])

Counter({'I': 17252, 'L': 6, 'N': 8, 'S': 1, 'T': 14, 'V': 179})

In [68]:
# Binarize
binarizers = joblib.load('models/binarizer.pkl')
binarizers

{'P10': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P11': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P12': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P13': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P14': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P15': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P16': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P17': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P18': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P19': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P2': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P20': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P21': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P22': LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
 'P23':

In [69]:
binarized = pd.DataFrame()
for col in alignment.columns:
    idx = 'P' + str(col)
    binarized = binarizers[idx].transform(alignment[col])

KeyError: 'P44'